In [5]:
import pandas as pd
import numpy as np
import glob
from nilearn import plotting
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import make_first_level_design_matrix
import nibabel as nib
import csv

In [7]:
def plot_interactive_floc(subj_csv, root, html_outdir):
    # Parameters
    nr_sub = 1
    n_scans = 188
    tr = 1.63
    nr_cond = 7
    epoch_duration = 6.0
    nr_cont_gp = 16
    alpha_list = [0.001, 0.01, 0.05]

    # Load data from the CSV
    with open(subj_csv, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # skip header

        for row in csvreader:
            subj_id = row[0]
            session = row[1]

            # Get the paths
            func_data_run_1 = glob.glob(f"{root}fMRIPrep/preproc_bold/sub-{subj_id}_ses-{session}*run-1*desc-preproc_bold.nii.gz")
            func_data_run_2 = glob.glob(f"{root}fMRIPrep/preproc_bold/sub-{subj_id}_ses-{session}*run-2*desc-preproc_bold.nii.gz")
            subj_T1 = glob.glob(f"{root}sMRIPrep/mni_t1/sub-{subj_id}_ses-{session}*space-*preproc_T1w.nii.gz")[0]
            conf_data_run_1 = glob.glob(f"{root}fMRIPrep/confounds/sub-{subj_id}_ses-{session}*_run-1_desc-confounds_timeseries.tsv")[0]
            conf_data_run_2 = glob.glob(f"{root}fMRIPrep/confounds/sub-{subj_id}_ses-{session}*_run-2_desc-confounds_timeseries.tsv")[0]
            onsets_dir = f"{root}fMRIPrep/events/sub-control_task-floc_run-1_events.tsv"

            # Create a list of dataframes with motion confounds and framewise disp
            motion_conf_all = []
            temp_df = pd.read_csv(conf_data_run_1, delimiter='\t')
            temp_df_2 = pd.read_csv(conf_data_run_2, delimiter='\t')
            sub_motion_params_1 = pd.DataFrame(temp_df, columns=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'framewise_displacement'])
            sub_motion_params_1['framewise_displacement'][0] = 0  # Change the first elem to 0 from Nan
            sub_motion_params_2 = pd.DataFrame(temp_df_2, columns=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'framewise_displacement'])
            sub_motion_params_2['framewise_displacement'][0] = 0  # Change the first elem to 0 from Nan
            motion_conf_all.append([sub_motion_params_1, sub_motion_params_2])

            # Get the event onsets
            events_allsub = []
            events_df = pd.read_csv(onsets_dir, sep='\t')
            events_allsub.append([events_df, events_df]) 

            # Concatenate run-1 and run-2 for each patient
            func_data = []
            func_data.append([image.concat_imgs(func_data_run_1, auto_resample=False), image.concat_imgs(func_data_run_2, auto_resample=False)])

            # Smoothing
            func_data_sm = [] 
            for n in range(nr_sub):
                func_data_sm.append(image.smooth_img(func_data[n], fwhm=4))

            frame_times = np.arange(n_scans) * tr
            design_matrices = []
            for n in range(nr_sub):
                design_matrices.append([make_first_level_design_matrix(frame_times, events=events_df, hrf_model='spm'), make_first_level_design_matrix(frame_times, events=events_df, hrf_model='spm')])

            # contrast matrix
            contrast_matrix = np.eye(design_matrices[0][0].shape[1])
            contrasts = dict([(column, contrast_matrix[i]) for i, column in enumerate(design_matrices[0][0].columns)])
            design_matrix = design_matrices[0][0].reset_index()

            fmri_glm_single_subj = FirstLevelModel(tr, noise_model='ar1', standardize=False, hrf_model='spm')
            fmri_glm_single_subj = fmri_glm_single_subj.fit(func_data_sm[0], events=events_allsub[0])
    
            contrast = (contrasts['Faces'], contrasts['Faces'])
            test_contrast = {"faces-oth": contrast}

            subj_T1_img = image.load_img(subj_T1)

            for alpha in alpha_list:
                for contrast_id, contrast_val in test_contrast.items():
                    z_map = fmri_glm_single_subj.compute_contrast(contrast_val, output_type='z_score')
                    z_img_thresh, thresholded = threshold_stats_img(z_map, alpha=alpha, height_control="fpr", cluster_threshold=20, two_sided=False)
                    
                    view = plotting.view_img(
                        z_img_thresh,
                        colorbar=True,
                        bg_img=subj_T1_img,
                        cut_coords=[-45, -57, -12])
                    
                    output_dir = html_outdir
                    view.save_as_html(f"{output_dir}/sub-{subj_id}_ses-{session}_alpha{alpha}_interactive_floc.html")

In [12]:
dyslexia_data = '/Volumes/language/language/Dyslexia_project/BIDS/derivatives/'
participants = '/Volumes/language/language/Dyslexia_project/BIDS/derivatives/fMRIPrep/participants_floc.csv'
output_matches = '/Volumes/language/language/Dyslexia_project/BIDS/derivatives/fMRIPrep/html'

plot_interactive_floc(subj_csv=participants, root=dyslexia_data, html_outdir=output_matches)

/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/jpillai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' wi